# 참고자료

https://colab.research.google.com/github/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb#scrollTo=Lf9uw2MKe-Yb

https://wikidocs.net/60314

https://wikidocs.net/64517

https://codlingual.tistory.com/91

# 0.라이브러리 import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext import data 
from torchtext.data import Field, BucketIterator
# from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

/home/dsail/hyolim/hyolim_py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 1. 전처리
## 1.1 tokenizer 만들기
* 토큰화
    * 주어진 텍스트를 단어 또는 문자 단위로 자르는 것
    * 영어의 경우, 토큰화 도구로 spacy와 NLTK를 주로 사용

* 설치 코드
    * python -m spacy download en
    * python -m spacy download de

In [3]:
# 각 언어에 맞는 tokenizer 불러오기 
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [4]:
# 문장을 토큰화 해주는 함수
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
# 영어 문장이 주어졌을 때 토큰화 되는 것을 확인 가능
text = "A Dog Run back corner near spare bedrooms"
print(tokenize_en(text))

['A', 'Dog', 'Run', 'back', 'corner', 'near', 'spare', 'bedrooms']


* 필드 정의하기(torchtext.data)
    * 앞으로 어떤 전처리를 할 것인지를 정의
    * tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
    * lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
    * batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)

In [6]:
SRC = data.Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = data.Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

## 1.2 Multi30k 데이터 로드
* 데이터셋 만들기
    * exts : 어떤 언어 사용할지 명시 (input 언어를 먼저 씀)
    * fields = (입력, 출력) 

In [7]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

print(f"학습 데이터셋(training dataset) 크기: {len(train_data.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_data.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_data.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [8]:
# 데이터 확인
print(vars(train_data.examples[0]))
# -> 여기서 독일어의 경우 단어 순서가 거꾸로 출력

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


* 단어 집합(vocabulary) 만들기
    * 토큰화 전처리를 끝낸 후, 각 단어에 고유한 정수를 맵핑해주는 정수 인코딩(Integer enoding) 작업이 필요. 이 전처리를 위해서 우선 단어 집합을 만들어야 하는데 이는 -> unique한 토큰들을 index에 대응시키는 과정
    * min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가. 2로 설정한 경우 한 번만 나오는 단어는 unk 토큰으로 처리

In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

# 생성된 단어 집합 크기 확인
print('단어 집합의 크기 : {}'.format(len(SRC.vocab)))
print('단어 집합의 크기 : {}'.format(len(TRG.vocab)))

단어 집합의 크기 : 7853
단어 집합의 크기 : 5893


* 생성된 단어 집합 내의 단어들 .stoi로 확인 가능
    * 토치텍스트가 임의로 특별 토큰인 'unk': 0, 'pad': 1를 추가함
    * unk는 단어 집합에 없는 단어를 표현할 때, pad는 길이를 맞추는 패딩 작업을 할 때 사용

In [10]:
print(SRC.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fe81329fa60>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '.': 4, 'ein': 5, 'einem': 6, 'in': 7, 'eine': 8, ',': 9, 'und': 10, 'mit': 11, 'auf': 12, 'mann': 13, 'einer': 14, 'der': 15, 'frau': 16, 'die': 17, 'zwei': 18, 'einen': 19, 'im': 20, 'an': 21, 'von': 22, 'sich': 23, 'dem': 24, 'mädchen': 25, 'junge': 26, 'vor': 27, 'zu': 28, 'steht': 29, 'männer': 30, 'sitzt': 31, 'hund': 32, 'den': 33, 'straße': 34, 'während': 35, 'gruppe': 36, 'hält': 37, 'spielt': 38, 'das': 39, 'hemd': 40, 'personen': 41, 'über': 42, 'drei': 43, 'eines': 44, 'frauen': 45, 'blauen': 46, 'neben': 47, 'ist': 48, 'kind': 49, 'roten': 50, 'weißen': 51, 'stehen': 52, 'sitzen': 53, 'menschen': 54, 'am': 55, 'aus': 56, 'spielen': 57, 'durch': 58, 'bei': 59, 'geht': 60, 'trägt': 61, 'fährt': 62, 'wasser': 63, 'um': 64, 'kinder': 65, 'kleines': 66, 'person': 67, 'macht': 68, 'springt': 69, 'kleiner': 70, 'schwarzen': 71, 

In [11]:
print(TRG.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fe81322f040>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, 'a': 4, '.': 5, 'in': 6, 'the': 7, 'on': 8, 'man': 9, 'is': 10, 'and': 11, 'of': 12, 'with': 13, 'woman': 14, ',': 15, 'two': 16, 'are': 17, 'to': 18, 'people': 19, 'at': 20, 'an': 21, 'wearing': 22, 'shirt': 23, 'young': 24, 'white': 25, 'black': 26, 'his': 27, 'while': 28, 'blue': 29, 'men': 30, 'red': 31, 'sitting': 32, 'girl': 33, 'boy': 34, 'dog': 35, 'standing': 36, 'playing': 37, 'group': 38, 'street': 39, 'down': 40, 'walking': 41, '-': 42, 'front': 43, 'her': 44, 'holding': 45, 'one': 46, 'water': 47, 'three': 48, 'by': 49, 'women': 50, 'up': 51, 'green': 52, 'little': 53, 'for': 54, 'child': 55, 'looking': 56, 'outside': 57, 'as': 58, 'large': 59, 'through': 60, 'brown': 61, 'yellow': 62, 'children': 63, 'person': 64, 'from': 65, 'their': 66, 'hat': 67, 'ball': 68, 'into': 69, 'small': 70, 'next': 71, 'other': 72, 'dressed':

* 데이터로더 만들기
    * 데이터로더: 데이터셋에서 미니 배치만큼 데이터를 로드하게 만들어주는 역할
    * Iterator로 데이터로더를 만듦.

In [12]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128 # 배치 크기(batch size): 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)

In [13]:
for i, batch in enumerate(train_iterator):

    src = batch.src
    trg = batch.trg
    
    print(f"첫 번째 배치 크기: {src.shape}")
    break # 첫 번째 배치만 확인

첫 번째 배치 크기: torch.Size([128, 23])


In [14]:
# src 정의하기
batch = next(iter(train_iterator))
print(type(batch))
print(batch)
src = batch.src
src
x=src
print(x.shape)
x.is_cuda
x

<class 'torchtext.data.batch.Batch'>

[torchtext.data.batch.Batch of size 128 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 128x33 (GPU 1)]
	[.trg]:[torch.cuda.LongTensor of size 128x35 (GPU 1)]
torch.Size([128, 33])


tensor([[ 2,  8, 67,  ...,  1,  1,  1],
        [ 2, 43, 41,  ...,  1,  1,  1],
        [ 2,  8, 16,  ...,  1,  1,  1],
        ...,
        [ 2,  8, 36,  ...,  1,  1,  1],
        [ 2,  5, 13,  ...,  1,  1,  1],
        [ 2,  8, 16,  ...,  1,  1,  1]], device='cuda:1')

# 2. model 만들기
그 전에 필요한 코드 먼저 돌리기

In [15]:
# 정의
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

input_dim=INPUT_DIM
hid_dim=HID_DIM
n_layers=ENC_LAYERS
n_heads=ENC_HEADS
pf_dim=ENC_PF_DIM
dropout=ENC_DROPOUT
device=device
max_length = 100 # position embedding의 'vocabulary' size가 100이어서 100 토큰 길이까지 문장을 수용함

In [16]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim,dropout, device):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device) # 멀티-헤드 어텐션
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout) # 위치인코더 -> 위치 인코딩       
        self.ff_layer_norm = nn.LayerNorm(hid_dim) # 덧셈과 계층 정규화 
        self.dropout = nn.Dropout(dropout)
          
    def forward(self, src, src_mask):
        _src, _ = self.self_attention(src, src, src, src_mask) ##########
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        _src = self.positionwise_feedforward(src)
        src = self.ff_layer_norm(src + self.dropout(_src))
        return src

In [17]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()        
        assert hid_dim % n_heads == 0    # assert: 가정 설정문, 뒤의 조건이 True가 아니면 error를 발생시킴
        self.hid_dim = hid_dim 
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads # d_model의 차원을 n_heads개로 나누어서 병렬 어텐션 수행.
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)        
        self.fc_o = nn.Linear(hid_dim, hid_dim) # WO(weight matrix)에 해당하는 밀집층 정의        
        self.dropout = nn.Dropout(dropout)        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)    
        
    def forward(self, query, key, value, mask = None): #######        
        batch_size = query.shape[0]                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale # Attention Energy 계산    
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        attention = torch.softmax(energy, dim = -1)
        x = torch.matmul(self.dropout(attention), V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(batch_size, -1, self.hid_dim)
        x = self.fc_o(x)
        return x, attention

In [18]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim) # fc를 두 번 거쳐서 입력과 출력 차원이 동일하도록(인풋의 shape를 그대로 유지함) 해줌        
        self.dropout = nn.Dropout(dropout)        
    def forward(self, x):
        x = self.dropout(torch.relu(self.fc_1(x)))
        x = self.fc_2(x)
        return x

In [19]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)       
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)        
        self.ff_layer_norm = nn.LayerNorm(hid_dim)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
        _trg = self.positionwise_feedforward(trg)
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        return trg, attention

## 2.1 인코더 아키텍처

In [20]:
class Encoder(nn.Module):
    def __init__(self,input_dim, hid_dim, n_layers, n_heads, pf_dim,dropout, device, max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim,dropout, device) for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        # src 문장의 embedding vector와 positional vector를 더해줌
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src # 마지막 레이어의 출력을 반환

def __init__(self,input_dim, hid_dim, n_layers, n_heads, pf_dim,dropout, device,max_length = 100):

In [21]:
# 정의
device = device
tok_embedding = nn.Embedding(input_dim, hid_dim).to(device) # Embedding(7853, 256)
pos_embedding = nn.Embedding(max_length, hid_dim).to(device) # Embedding(100, 256)
layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim,0.1, device) for _ in range(n_layers)]).to(device) 
dropout = nn.Dropout(0.1).to(device) # Dropout(p=0.1, inplace=False)
scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device) # tensor([16.])

* 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정    
    def make_src_mask(self, src):

In [22]:
# src_mask 정의하기
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
src_pad_idx=SRC_PAD_IDX
SRC_PAD_IDX

1

In [23]:
print(src.shape)
src # PAD가 1

torch.Size([128, 33])


tensor([[ 2,  8, 67,  ...,  1,  1,  1],
        [ 2, 43, 41,  ...,  1,  1,  1],
        [ 2,  8, 16,  ...,  1,  1,  1],
        ...,
        [ 2,  8, 36,  ...,  1,  1,  1],
        [ 2,  5, 13,  ...,  1,  1,  1],
        [ 2,  8, 16,  ...,  1,  1,  1]], device='cuda:1')

* 언스퀴즈(Unsqueeze) -특정 위치에 1인 차원을 추가하기
    * https://wikidocs.net/52846

In [24]:
# 예>
ft = torch.Tensor([0, 1, 2])
print(ft)
print(ft.shape) # 차원이 1개인 1차원 벡터

# 첫번째 차원에 1인 차원을 추가하기. 
# 첫번째 차원의 인덱스를 의미하는 숫자 0을 인자로 넣으면 첫번째 차원에 1인 차원이 추가됨
print(ft.unsqueeze(0)) # 인덱스가 0부터 시작해서 0은 첫번째 차원임.
print(ft.unsqueeze(0).shape) #  (3,)의 크기를 가졌던 1차원 벡터가 (1, 3)의 2차원 텐서로 변경

tensor([0., 1., 2.])
torch.Size([3])
tensor([[0., 1., 2.]])
torch.Size([1, 3])


In [25]:
# 변경해서 shape 확인해보기 
print((src != src_pad_idx).unsqueeze(0).shape)
print((src != src_pad_idx).unsqueeze(1).shape)

print((src != src_pad_idx).unsqueeze(0).unsqueeze(1).shape)

torch.Size([1, 128, 33])
torch.Size([128, 1, 33])
torch.Size([1, 1, 128, 33])


In [26]:
# src_mask 정의하기
src_mask = (src != src_pad_idx).unsqueeze(1).unsqueeze(2)
print(src_mask.shape)  #src_mask = [batch size, 1, 1, src len]
src_mask

torch.Size([128, 1, 1, 33])


tensor([[[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        ...,


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]]], device='cuda:1')

In [27]:
# 어떻게 생겼는지 확인해보기
src_mask[1]

tensor([[[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True, False, False, False, False, False, False, False,
          False, False, False]]], device='cuda:1')

def forward(self, src, src_mask):

In [28]:
# forward 진행 전에 shape 맞는지 확인하기
print(src.shape)        #src = [batch size, src len]
print(src_mask.shape)   #src_mask = [batch size, 1, 1, src len]

torch.Size([128, 33])
torch.Size([128, 1, 1, 33])


In [29]:
# 1.
batch_size = src.shape[0] # 128
print(batch_size)
src_len = src.shape[1]
print(src_len)

128
33


In [30]:
# pos 정의하기 전에 구조 확인하기
print(torch.arange(0, src_len).shape)
print(torch.arange(0, src_len))

print(torch.arange(0, src_len).unsqueeze(0).shape) # unsqueeze(0)을 추가하면
print(torch.arange(0, src_len).unsqueeze(0))

print(torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).shape) # repeat(batch_size, 1)을 추가하면
print(torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1))

torch.Size([33])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])
torch.Size([1, 33])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]])
torch.Size([128, 33])
tensor([[ 0,  1,  2,  ..., 30, 31, 32],
        [ 0,  1,  2,  ..., 30, 31, 32],
        [ 0,  1,  2,  ..., 30, 31, 32],
        ...,
        [ 0,  1,  2,  ..., 30, 31, 32],
        [ 0,  1,  2,  ..., 30, 31, 32],
        [ 0,  1,  2,  ..., 30, 31, 32]])


* torch에서의 repeat

In [31]:
# 예>
a = torch.tensor([1, 2, 3])
print(a)
a.repeat(4, 2)

tensor([1, 2, 3])


tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3]])

In [32]:
# 2. 
pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(device)
print(pos.shape) #pos = [batch size, src len]

torch.Size([128, 33])


https://moondol-ai.tistory.com/460

* 트랜스포머는 단어 입력을 순차적으로 받지 않아서, 단어의 위치 정보를 알려줄 필요가 있다. 이때, 각 단어의 임베딩 벡터에 위치 정보들을 더해 모델의 입력으로 쓰는데 이것이 포지셔널 인코딩이다.
* 입력으로 사용되는 임베딩 벡터들이 트랜스포머에 입력으로 쓰이기 전에 포지셔널 인코딩 값이 더해진다
* 결론적으로 임베딩 벡터와 포지셔널 인코딩의 덧셈은 사실, 임베딩 벡터가 모여 만들어진 문장 벡터 행렬과 포지셔널 인코딩 행렬의 덧셈연산으로 이루어지는 것이다

In [33]:
print(pos_embedding(pos).shape)
print(tok_embedding(src).shape)

torch.Size([128, 33, 256])
torch.Size([128, 33, 256])


In [34]:
# 3.
# src 문장의 embedding vector와 positional vector를 더해줌
src = dropout((tok_embedding(src) * scale) + pos_embedding(pos))
src.shape #src = [batch size, src len, hid dim]
# multihead attention layer에서 nn.linear에서 input 차원과 같도록 shape를 조정해줌 torch.Size([batch size, src len, 256 -> hid_dim])

torch.Size([128, 33, 256])

In [35]:
# 4. 
# 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
for layer in layers:
    src = layer(src, src_mask)
src # 마지막 레이어의 출력을 반환
src.shape #src = [batch size, src len, hid dim]

torch.Size([128, 33, 256])

## 2.2 MultiHeadAttentionLayer
* 한 번의 어텐션보다 어텐션을 병렬로 여러번 사용하는 것이 더 효과적일 것이다!
* 어텐션(attention)의 세 입력 요소
    * 쿼리(queries), 키(keys), 값(values)
* 하이퍼 파라미터(hyperparameter)
    * hidden_dim: 하나의 단어에 대한 임베딩 차원
    * n_heads: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * dropout_ratio: 드롭아웃(dropout) 비율
    
- h=8 parallel attention layers or heads
- 각 헤드마다 dk = dv = dmodel/h = 64(512/8)의 차원을 설정.

- encoderlayer에서 self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device) 다음과 같이 쓰임

In [36]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0    # assert: 가정 설정문, 뒤의 조건이 True가 아니면 error를 발생시킴
        
        self.hid_dim = hid_dim 
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads # d_model의 차원을 n_heads개로 나누어서 병렬 어텐션 수행.
        
        # WQ, WK, WV에 해당하는 밀집층 정의
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim) # WO(weight matrix)에 해당하는 밀집층 정의
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None): #######
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
        
        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도, permute를 통해 순서를 변경, scaled dot product 연산시 데이터 입력 순서가 달라서
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
        
        # Q와 K에 대해서 행렬곱해준 후 scale 진행
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale # Attention Energy 계산    
        
        #energy = [batch size, n heads, query len, key len]
        
        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기, 입력 문장에 <PAD>토큰이 있을 경우 어텐션에서 제외하기 위한 연산 -> <PAD>의 경우, 실질적인 의미를 가진 단어가 아니어서 유사도를 구하지 않도록 마스킹해줌.
            energy = energy.masked_fill(mask == 0, -1e10)
        
        # 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
        
        # Scaled Dot-Product Attention을 계산
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

def __init__(self, hid_dim, n_heads, dropout, device):

In [37]:
# 정의
assert hid_dim % n_heads == 0  

hid_dim = hid_dim # 임베딩 차원, 256
n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수, 8
head_dim = hid_dim // n_heads # 각 헤드(head)에서의 임베딩 차원, 32

# 선형회귀모델 nn.Linear(input_dim,output_dim)
fc_q = nn.Linear(hid_dim, hid_dim).to(device) # Query 값에 적용될 FC 레이어, Linear(in_features=256, out_features=256, bias=True)
fc_k = nn.Linear(hid_dim, hid_dim).to(device) # Key 값에 적용될 FC 레이어, Linear(in_features=256, out_features=256, bias=True)
fc_v = nn.Linear(hid_dim, hid_dim).to(device) # Value 값에 적용될 FC 레이어, Linear(in_features=256, out_features=256, bias=True)
fc_o = nn.Linear(hid_dim, hid_dim).to(device) # Linear(in_features=256, out_features=256, bias=True)

dropout = nn.Dropout(0.1).to(device) # Dropout(p=0.1, inplace=False)

scale = torch.sqrt(torch.FloatTensor([head_dim])).to(device)

def forward(self, query, key, value, mask = None):

In [38]:
# query, key, value, mask = None 정의하기
query=src
print(query.shape) #query = [batch size, query len, hid dim]
key=src
print(key.shape) #key = [batch size, key len, hid dim]
value=src
print(value.shape) #value = [batch size, value len, hid dim]
mask = None

torch.Size([128, 33, 256])
torch.Size([128, 33, 256])
torch.Size([128, 33, 256])


def forward(self, query, key, value, mask = None):

In [39]:
# 1.
batch_size = query.shape[0]
batch_size # 128

128

In [40]:
# 2.
Q = fc_q(query)
print(Q.shape) #Q = [batch size, query len, hid dim]
print(Q)
K = fc_k(key)
print(K.shape) #K = [batch size, key len, hid dim]
V = fc_v(value)
print(V.shape) #V = [batch size, value len, hid dim]      

torch.Size([128, 33, 256])
tensor([[[-3.0326e-01, -2.3057e-03, -9.2200e-02,  ...,  7.8596e-01,
          -1.4714e-01, -2.8933e-03],
         [ 8.3807e-02, -5.6867e-01, -1.1365e+00,  ...,  6.9740e-01,
           1.1024e+00,  3.5360e-01],
         [-2.7398e-01, -8.3326e-01, -7.3765e-01,  ...,  5.2766e-01,
           2.0044e-01, -4.3675e-01],
         ...,
         [-1.0129e+00,  8.9158e-02, -1.6086e-01,  ...,  5.1141e-01,
          -3.5899e-01, -3.8408e-01],
         [-1.0339e+00,  5.2001e-01, -5.7130e-02,  ...,  5.8669e-01,
          -7.9526e-01, -1.9262e-01],
         [-9.4813e-01,  7.5911e-01, -4.1976e-01,  ...,  7.6736e-01,
          -6.2036e-01, -4.0017e-01]],

        [[-1.3844e-01,  2.0960e-01, -9.2511e-02,  ...,  8.4334e-01,
          -1.8276e-02,  3.1615e-01],
         [-6.5478e-01,  2.6433e-01,  5.1745e-01,  ...,  6.0186e-02,
          -7.7943e-01,  1.8775e-01],
         [ 9.4951e-01, -1.2799e-01, -2.8777e-01,  ...,  3.0459e-01,
           6.5994e-01, -3.8879e-01],
         ...

* pytorch view
    * 기본적으로 변경 전과 변경 후의 텐서 안의 원소의 개수가 유지되어야 함

In [41]:
Q.view(batch_size, -1, n_heads, head_dim).shape # 256이 8, 32로 나누어짐

torch.Size([128, 33, 8, 32])

* pytorch permutate
    * 순서 바뀜

In [42]:
print(Q.view(batch_size, -1, n_heads, head_dim).permute(0, 2, 1, 3).shape)
# 0,1,2,3 이 0,2,1,3 으로 가운데 두 값의 순서가 바뀜
print(Q.view(batch_size, -1, n_heads, head_dim).permute(3, 2, 1, 0).shape)

torch.Size([128, 8, 33, 32])
torch.Size([32, 8, 33, 128])


* multi-head attention
    * hidden_dim(256) → n_heads(8) X head_dim(32) 형태로 변형
    * n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도     
    * permute를 통해 순서를 변경, scaled dot product 연산시 데이터 입력 순서가 달라서

In [43]:
# 3.
Q = Q.view(batch_size, -1, n_heads, head_dim).permute(0, 2, 1, 3)
print(Q.shape) #Q = [batch size, n heads, query len, head dim]
K = K.view(batch_size, -1, n_heads, head_dim).permute(0, 2, 1, 3)
print(K.shape) #K = [batch size, n heads, key len, head dim]
V = V.view(batch_size, -1, n_heads, head_dim).permute(0, 2, 1, 3)
print(V.shape) #V = [batch size, n heads, value len, head dim]

torch.Size([128, 8, 33, 32])
torch.Size([128, 8, 33, 32])
torch.Size([128, 8, 33, 32])


In [44]:
print(Q.shape)
print(K.permute(0, 1, 3, 2).shape) # 행렬곱 위해서 순서 바꿈(permutate 사용 이유)

torch.Size([128, 8, 33, 32])
torch.Size([128, 8, 32, 33])


1.2.1 Scaled Dot-Product Attention
* scaling
    * attention function으로서 dot-product attention을 채택.
        * (더 빠르고 공간효율적)
    * 이 때, 1/√dk로 scaling 해줌 -> 모든 값들이 0 근처에 옴
 * softmax
     *  values에 대한 weights를 얻고자 softmax function 시행

In [45]:
# 4. 
# Q와 K에 대해서 행렬곱해준 후 scale 진행, attention energy 계산
energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / scale 
print(energy.shape) #energy = [batch size, n heads, query len, key len]

torch.Size([128, 8, 33, 33])


In [46]:
# 5. 
# 마스크(mask)를 사용하는 경우
if mask is not None:
    # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기, 입력 문장에 <PAD>토큰이 있을 경우 어텐션에서 제외하기 위한 연산 -> <PAD>의 경우, 실질적인 의미를 가진 단어가 아니어서 유사도를 구하지 않도록 마스킹해줌.
    energy = energy.masked_fill(mask == 0, -1e10)

In [47]:
# 6. 
# 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
attention = torch.softmax(energy, dim = -1)  
print(attention.shape) #attention = [batch size, n heads, query len, key len]

torch.Size([128, 8, 33, 33])


In [48]:
# 행렬곱
print(dropout(attention).shape)
print(V.shape)

torch.Size([128, 8, 33, 33])
torch.Size([128, 8, 33, 32])


In [49]:
print(x.shape) # x=[batch size, src len] __ 아예 처음 불러온 값

# 7. 
# Scaled Dot-Product Attention을 계산
x = torch.matmul(dropout(attention), V)
print(x.shape) #x = [batch size, n heads, query len, head dim]

torch.Size([128, 33])
torch.Size([128, 8, 33, 32])


## contiguous 더 자세하게 . 다른 것과 비교도 같이

* contiguous
    * 메모리 저장 상태를 바꾸는 것
    * 새로운 메모리 공간에 데이터를 복사해 주소값 연속성을 가변적이게 만들어줌

In [50]:
# 8.
x = x.permute(0, 2, 1, 3).contiguous()  #x = [batch size, query len, n heads, head dim]
print(x.shape)

torch.Size([128, 33, 8, 32])


In [51]:
# 9.
x = x.view(batch_size, -1, hid_dim) 
print(x.shape) #x = [batch size, query len, hid dim]

torch.Size([128, 33, 256])


In [52]:
# 10.
x = fc_o(x) # 선형회귀모델 거침
print(x.shape) #x = [batch size, query len, hid dim]

torch.Size([128, 33, 256])


In [53]:
# 최종 return 값
print(x.shape)
print(attention.shape)

torch.Size([128, 33, 256])
torch.Size([128, 8, 33, 33])


## 2.4 PositionwiseFeedforwardLayer

* 인코더, 디코더에서 공통적으로 가지는 층으로 완전 연결(FFNN, Fully-connected)을 적용하는 부분으로 position(각 단어)마다 적용되기 때문에 position-wise라 불림

- encoder layer에서 self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)

- 2개의 FC layer를 가짐
- 하이퍼 파라미터(hyperparameter)
    * hidden_dim: 하나의 단어에 대한 임베딩 차원
    * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
    * dropout_ratio: 드롭아웃(dropout) 비율
- Multi-Head Attention Layer의 output을 input으로 받아 연산을 수행하고, 다음 Encoder에게 output을 넘겨줌. 첫번째 FC Layer의 output에 ReLU를 적용

In [54]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim) # fc를 두 번 거쳐서 입력과 출력 차원이 동일하도록(인풋의 shape를 그대로 유지함) 해줌
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

 def __init__(self, hid_dim, pf_dim, dropout):

In [55]:
# 정의
fc_1 = nn.Linear(hid_dim, pf_dim).to(device) # Linear(in_features=256, out_features=512, bias=True)
fc_2 = nn.Linear(pf_dim, hid_dim).to(device) # Linear(in_features=512, out_features=256, bias=True)
dropout = nn.Dropout(0.1).to(device) # Dropout(p=0.1, inplace=False)

In [56]:
# Multi-Head Attention Layer의 output을 input으로 받아 연산을 수행
print(x.shape) #x = [batch size, seq len, hid dim] , multihead_atttention을 거쳐 나온 x값

torch.Size([128, 33, 256])


def forward(self, x):

In [57]:
fc_1(x)

tensor([[[ 1.2092e-01, -9.6780e-03,  2.1375e-01,  ..., -4.0775e-02,
          -1.6421e-02,  9.6035e-02],
         [ 1.2328e-01, -9.5611e-03,  1.8613e-01,  ..., -5.3108e-02,
           1.1142e-03,  1.3111e-01],
         [ 9.8520e-02, -3.4071e-03,  1.9743e-01,  ..., -6.5517e-02,
           3.8334e-02,  8.0034e-02],
         ...,
         [ 9.6879e-02, -3.0070e-02,  1.9713e-01,  ..., -7.1215e-02,
           3.9438e-02,  8.9874e-02],
         [ 1.1846e-01, -3.4520e-02,  2.3427e-01,  ..., -7.9383e-02,
           1.7571e-02,  9.7180e-02],
         [ 1.1914e-01, -3.8673e-02,  2.4439e-01,  ..., -9.1326e-02,
           3.5123e-02,  1.1855e-01]],

        [[ 2.8158e-02,  2.5383e-02,  1.6788e-01,  ..., -3.6718e-02,
           1.3788e-01,  8.7554e-03],
         [-2.6828e-04,  2.3287e-02,  1.7942e-01,  ..., -7.9074e-02,
           1.2574e-01,  2.6135e-02],
         [ 3.9010e-02, -3.5542e-02,  1.8451e-01,  ..., -3.4474e-02,
           1.2946e-01, -1.7323e-02],
         ...,
         [ 2.9507e-02, -1

In [58]:
torch.relu(fc_1(x)) # relu 시행시 값이 어떻게 바뀌는지 확인

tensor([[[1.2092e-01, 0.0000e+00, 2.1375e-01,  ..., 0.0000e+00,
          0.0000e+00, 9.6035e-02],
         [1.2328e-01, 0.0000e+00, 1.8613e-01,  ..., 0.0000e+00,
          1.1142e-03, 1.3111e-01],
         [9.8520e-02, 0.0000e+00, 1.9743e-01,  ..., 0.0000e+00,
          3.8334e-02, 8.0034e-02],
         ...,
         [9.6879e-02, 0.0000e+00, 1.9713e-01,  ..., 0.0000e+00,
          3.9438e-02, 8.9874e-02],
         [1.1846e-01, 0.0000e+00, 2.3427e-01,  ..., 0.0000e+00,
          1.7571e-02, 9.7180e-02],
         [1.1914e-01, 0.0000e+00, 2.4439e-01,  ..., 0.0000e+00,
          3.5123e-02, 1.1855e-01]],

        [[2.8158e-02, 2.5383e-02, 1.6788e-01,  ..., 0.0000e+00,
          1.3788e-01, 8.7554e-03],
         [0.0000e+00, 2.3287e-02, 1.7942e-01,  ..., 0.0000e+00,
          1.2574e-01, 2.6135e-02],
         [3.9010e-02, 0.0000e+00, 1.8451e-01,  ..., 0.0000e+00,
          1.2946e-01, 0.0000e+00],
         ...,
         [2.9507e-02, 0.0000e+00, 1.7448e-01,  ..., 0.0000e+00,
          1.257

In [59]:
# 1.
x = dropout(torch.relu(fc_1(x)))
x.shape #x = [batch size, seq len, pf dim]

torch.Size([128, 33, 512])

In [60]:
# 2.
x = fc_2(x)
# fc를 두 번 거쳐서 입력과 출력 차원이 동일하도록(인풋의 shape를 그대로 유지함) 해줌
       
# return 값
x.shape #x = [batch size, seq len, hid dim]

torch.Size([128, 33, 256])

## 2.5 인코더레이어
* 하나의 인코더 레이어에 대해
    * 입력, 출력의 차원이 같음.
    * -> 이를 이용해 트랜스포머의 인코더는 인코더 레이어를 여러 번 중첩해 사용.
* 역할
    * 인풋으로 들어오는 벡터에 대해 더 높은 차원(넓은 관점)에서의 context를 담음
* 하이퍼 파라미터(hyperparameter)
    * hidden_dim: 하나의 단어에 대한 임베딩 차원
    * n_heads: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
    * dropout_ratio: 드롭아웃(dropout) 비율
* pad 토큰에 대하여 마스크(mask) 값을 0으로 설정.

In [61]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim,dropout, device):
        super().__init__()
        
        ''' Multi Head self-Attention '''   
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device) # 멀티-헤드 어텐션
        
        ''' Positional FeedForward Layer'''
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout) # 위치인코더 -> 위치 인코딩
        
        self.ff_layer_norm = nn.LayerNorm(hid_dim) # 덧셈과 계층 정규화 
        
        self.dropout = nn.Dropout(dropout)
        
    # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식    
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        # 필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절 가능
        _src, _ = self.self_attention(src, src, src, src_mask) ##########
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

def __init__(self, hid_dim, n_heads, pf_dim,dropout, device):

In [62]:
# 정의
self_attn_layer_norm = nn.LayerNorm(hid_dim).to(device) # LayerNorm((256,), eps=1e-05, elementwise_affine=True)
ff_layer_norm = nn.LayerNorm(hid_dim).to(device) # LayerNorm((256,), eps=1e-05, elementwise_affine=True)
self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, 0.1, device).to(device)
positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, 0.1).to(device)
dropout = nn.Dropout(0.1).to(device)

def forward(self, src, src_mask):

In [63]:
# forward 시행 전 shape 확인
print(src.shape)        #src = [batch size, src len, hid dim]
print(src_mask.shape)        #src_mask = [batch size, 1, 1, src len] 

torch.Size([128, 33, 256])
torch.Size([128, 1, 1, 33])


In [64]:
src

tensor([[[ 0.1421,  2.1862,  1.2786,  ...,  0.8169, -0.8156,  0.9939],
         [ 0.1398, -0.7816, -0.4919,  ...,  0.4420,  1.0236,  0.8938],
         [-0.8042,  1.5627,  1.3480,  ..., -1.2275,  0.6387,  0.1244],
         ...,
         [-0.4649,  0.1771,  0.2606,  ...,  0.6818, -0.8742, -0.6000],
         [-1.1260,  0.1893,  0.3192,  ...,  1.1765, -1.0056, -0.6380],
         [-0.8556,  0.7348,  0.4981,  ...,  0.9201, -0.4887, -0.6634]],

        [[ 1.0013,  1.4606,  1.0953,  ...,  1.2089, -0.4154,  0.4112],
         [-0.4665,  0.5787,  1.9497,  ...,  0.4934,  0.3069, -1.0587],
         [ 0.0818,  2.4000,  1.0493,  ..., -0.1271,  1.0192,  1.4516],
         ...,
         [-2.2811, -0.2598,  0.8500,  ...,  0.7258, -0.7214, -0.2559],
         [-1.5202,  0.1499,  0.4923,  ...,  1.3702, -0.6123, -1.1193],
         [-1.9052,  0.2877,  0.6039,  ...,  1.2162, -0.1249, -0.8511]],

        [[ 0.1270,  2.1402,  0.1093,  ...,  1.7408,  0.6339,  0.6054],
         [ 0.8079, -0.6077,  0.4818,  ..., -0

In [65]:
src_mask

tensor([[[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        ...,


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]]], device='cuda:1')

* self attention
* class MultiHeadAttentionLayer(nn.Module):
    * def __init__(self, hid_dim, n_heads, dropout, device):
    
* class EncoderLayer(nn.Module):
    * def __init__(self, hid_dim, n_heads, pf_dim,dropout, device):
        * self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)

* 인코더의 셀프어텐션
    * Query =Key= Value가 모두 같음(모두 인코더에서 가져옴)
    * 필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절 가능 -- src_mask라고 뒤에 설정하는 이유
    * -> src, src, srcf라고 둠

In [66]:
self_attention

MultiHeadAttentionLayer(
  (fc_q): Linear(in_features=256, out_features=256, bias=True)
  (fc_k): Linear(in_features=256, out_features=256, bias=True)
  (fc_v): Linear(in_features=256, out_features=256, bias=True)
  (fc_o): Linear(in_features=256, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [67]:
# 1.
_src, _ = self_attention(src, src, src, src_mask)
print(_src.shape)

torch.Size([128, 33, 256])


In [68]:
_src

tensor([[[-0.0707,  0.2216, -0.1377,  ..., -0.2040,  0.0182, -0.3989],
         [-0.0554,  0.2112, -0.0661,  ..., -0.1924,  0.1150, -0.3144],
         [-0.0478,  0.1968, -0.0787,  ..., -0.1731, -0.0521, -0.3144],
         ...,
         [-0.1214,  0.2350, -0.1124,  ..., -0.2113,  0.0686, -0.3812],
         [-0.1392,  0.2145, -0.0693,  ..., -0.2185,  0.0169, -0.3544],
         [-0.1021,  0.2488, -0.1169,  ..., -0.1837,  0.0098, -0.3277]],

        [[ 0.0233,  0.1583,  0.0265,  ..., -0.2480,  0.1666, -0.3592],
         [ 0.0411,  0.2017,  0.0151,  ..., -0.2439,  0.1154, -0.2769],
         [ 0.0393,  0.1774,  0.0433,  ..., -0.2254,  0.0909, -0.2616],
         ...,
         [ 0.0530,  0.1711, -0.0253,  ..., -0.2256,  0.0871, -0.2711],
         [ 0.0674,  0.1612, -0.0396,  ..., -0.2035,  0.0803, -0.2230],
         [ 0.0701,  0.1705, -0.0051,  ..., -0.2236,  0.0587, -0.2890]],

        [[ 0.0201,  0.1924, -0.0083,  ..., -0.3083,  0.2567, -0.0573],
         [ 0.0083,  0.2459,  0.0486,  ..., -0

In [69]:
self_attn_layer_norm

LayerNorm((256,), eps=1e-05, elementwise_affine=True)

In [70]:
# 2.
# 드롭아웃, 잔차연결, 정규화 진행
src = self_attn_layer_norm(src + dropout(_src)) # LayerNorm((256,), eps=1e-05, elementwise_affine=True)
print(src.shape) #src = [batch size, src len, hid dim]

torch.Size([128, 33, 256])


* position-wise feed forward
    * 인풋, 아웃풋 차원은 256
    * attention과 함께 쓰이는 fully connected feed-forward network

In [71]:
positionwise_feedforward

PositionwiseFeedforwardLayer(
  (fc_1): Linear(in_features=256, out_features=512, bias=True)
  (fc_2): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [72]:
# 3. 
# positionwise feedforward
_src = positionwise_feedforward(src)
print(_src.shape) 

torch.Size([128, 33, 256])


In [73]:
ff_layer_norm

LayerNorm((256,), eps=1e-05, elementwise_affine=True)

In [74]:
# 4. 
# 드롭아웃, 잔차연결, 정규화 진행
src = ff_layer_norm(src + dropout(_src))

# return
print(src.shape) #src = [batch size, src len, hid dim]

torch.Size([128, 33, 256])
